# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s): Jonathan Reich
###### Python version: 3.5.6,
###### Submission deadline: 5 pm, Mon 8 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [478]:
from collections import defaultdict
import copy
import operator
import random
import math
import numpy as np
import os
import sys

In [343]:
# This function should open a data file in csv, and transform it into a usable format 
def preprocess(file, split=0.0):
    """Opens a csv file of data and optionally transforms the data into 'training' and 'test' splits for
    'holdout' strategy. Also generates a dictionary structure that
    summarises the statistics about the number of instances of individual classes, as well as the number of instances 
    found in the complete dataset, and the number of attributes found in the dataset.
    
    Args:
        file (csv file): dataset
        split (number): optional number that splits the data into partitions; default is set to 0 for training on all data.
    
    Returns:
        class_dict (dic): the number of instances found in the complete dataset.
        cleaned_data_train (list): two-dimensional list of lists containing the training data.
        cleaned_data_train (list): two-dimensional list of lists containing the test data.
        n_instances_test (number) : the number of instances found in the test dataset.
        class_dict (dic): dictionary giving counts of individual classes for the training dataset.
        test_dict (dic): dictionary giving counts of individual classes for the test dataset
        n_instances_tr (number): integer of the number of instances found in the training dataset.
        n_attrs (number): the number of attributes/properties found in a given instance.
    """
    
    n_attrs = 0
    f = open(file, 'r')
    # remember start of dataset for counting purposes.
    x = f.tell()
    first_line = f.readline().split(',')
    # read off number of attributes.
    for line in first_line[:-1]:
        n_attrs+=1
    # seek back to first line of dataset for counting purposes.
    f.seek(x)
    g = list(f)
    f.close()
    # count number of instances in whole dataset.
    n_instances = len(g)
    # if we want to use the 'holdout' strategy:
    if split > 0:
        cleaned_data_train = []
        cleaned_data_test = []
        test_dict=defaultdict(int)
        train_dict=defaultdict(int)
        # randomise the data before partitioning.
        random.shuffle(g)
        n_instances_tr = int(n_instances*split)
        # training instances split.
        training_data = g[:n_instances_tr]
        # test instance split.
        test_data = g[n_instances_tr:]
        n_instances_test = len(test_data)
        for line in training_data:
            # count classes in training set.
            train_dict[line.strip().split(",")[-1]] += 1
            # list of cleaned training data.
            cleaned_data_train.append(line.strip().split(','))
        for line in test_data:
            # count classes in test set.
            cleaned_data_test.append(line.strip().split(','))
            # list of cleaning test data.
            test_dict[line.strip().split(",")[-1]] += 1
        return cleaned_data_train, cleaned_data_test, n_instances_test, train_dict, test_dict, n_instances_tr, n_attrs
    
    cleaned_data = []
    
    class_dict=defaultdict(int)
    
    # No partitiong of data, so we will train on the full dataset:
    for line in g:
        # count classes in full dataset.
        class_dict[line.strip().split(",")[-1]] += 1
        # append the cleaned data to a list for further processing.
        cleaned_data.append(line.strip().split(','))
   
    return class_dict, n_instances, cleaned_data, n_attrs
        

In [580]:
# This function should build a supervised NB model
def train(training_data, n_instances, class_dict, n_attrs, ig_dic=False):
    """Trains the data from a preprocessed list and returns dictionaries of probabilities used to predict instances.
    
    Args:
        training_data (list): preprocessed dataset.
        n_instances (number): number of instances found in dataset.
        class_dict (dic): counts of classes.
        n_attrs (number): number of attributes found in a given instance.
        ig_dic (dictionary): gives a summary of counts of individual attributes (for use for information gain metric).
    
    Returns:
        prior (dic): dictionary of prior probabilities of classes.
        post (dic): dictionary of posterior probabilities of classes.
        training_data (list): training data in preprocessed form for next stage of the pipeline.
    """
    
    
    prior = defaultdict(int)
    # store the prior probabilities a dictionary.
    for key,val in class_dict.items():
        prior[key] = val/n_instances
    
    post=defaultdict(int)
    
    # iterate through the number of attributes.
    for attr in range(n_attrs):
        # create a nested dictionary for each attribute.
        temp_dict = defaultdict(int)
        for line in training_data:
            cls = line[-1]
            # create a key for each class in the nested dictionary.
            if cls not in temp_dict:
                temp_dict[cls] = defaultdict(int)
            
            # if the attribute itself doesn't already exist in the dictionary, count it as one.
            if line[attr] not in temp_dict[cls]:
                temp_dict[cls][line[attr]] = 1
            # otherwise, increment the count (this class, attribute pair already exists.)
            else:
                temp_dict[cls][line[attr]] += 1
        
        # set the posteior attribute to the counter from the 'temp_dict' object.
        post[attr] = temp_dict
        
        
        # iterate through each class in the dataset.
        for cls in prior.keys():
            # find aggregate of the values for each class.
            sum_value = sum(temp_dict[cls].values())
            for key, val in temp_dict[cls].items():
                # if parameter 'ig_dic' is true, then we will use the 'post' dic for
                # raw counts (to be used for the information gain function)
                if ig_dic:
                    temp_dict[cls][key] = val
                # Divide the class and attribute pair by the sum of the overall class.
                else:
                    temp_dict[cls][key] = val/sum_value

    return prior, post, training_data

In [581]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(prior, post, data):
    """Given instances of classes, predicts the class of each instance using the Naive Bayes' formula.
    
    Args:
        prior (dic): dictionary of prior probabilities of classes.
        post (dic): dictionary of posterior probabilities of classes.
        data (list): data on which to make predictions w/r/t the classes.
    
    Returns:
        final (dic): dictionary containing the classes of each predicted instance along with their associated probabilities.
        data (list): passes back data for use in evaluation metrics.
    """
    # value to be used to ensure probabilities don't hit 'zero', if a given attribute doesn't exist.
    EPSILON = 0.000001
    final = defaultdict(int)
    n_instances = len(data)
    # iterate over the data and create nested dictionaries to store predictions.
    for i, ins in enumerate(data):
        pred = defaultdict(int)
        for key, val in prior.items():
            # the prediction for the data point needs to be multiplied by the prior and posterior probabilities, 
            # according to Bayes' formula.
            pred[key] = val
            for j in range(len(ins)-1):
                att = ins[j]
                # we will treat '?' as missing data.
                if att != '?':
                    # if the attribute isn't 'zero' or a '?' then multiply our prediction probability by the next entry.
                    if att in post[j][key]:
                        pred[key] *= post[j][key][att]
                    # otherwise, multiply our prediction probability by an infinitesimally small value.
                    else:
                        pred[key] *= EPSILON/n_instances
        # the predicted class for the instance will be the class that has the largest probability after looping through 
        # the inner loops.
        final[i] = max(pred.items(), key=operator.itemgetter(1))
    
    return final, data

In [582]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate(file, holdout_split=0):
    """Given a csv file, runs through the pipeline of preprocessing, training, predicting, then evaluating the dataset to 
    obtain various metrics such as the overall accuracy (disregarding individual class accuracy), and the precision, recall
    and F1-score for each class. Also returns a 'ZeroR' Baseline.
    
    Args:
        file (csv file): dataset
        holdout_split (number): optional kwarg that splits the data into training and test partitions.
    
    Returns:
        accuracy (number): a numeric value that represents the ratio of the number of instances that matched (true positives)
        from the original/test dataset over the number of instances found in the set.
        baseline (number): 'zeroR' metric used as a baseline that classes each instant according to the majority class found in
        the dataset.
        confusion (dictionary of dictionaries): Summarises for each individual class in the dataset the precision, recall and 
        f1-score (the harmonic mean of precision and recall).
    """
    
    # sanity to check to ensure the holdout split is a valid value.
    assert (holdout_split >=0 and holdout_split <=1)
    if holdout_split > 0:
        # get preprocessed data.
        cleaned_data_train, cleaned_data_test, n_instances, dic, test_dict, first_split, n_attrs = preprocess(file, split=holdout_split)
        # get prior and post probability dictionary objects.
        prior,post,_ = train(cleaned_data_train, first_split, dic, n_attrs)
        # get the predicted classes, and get back the raw data in a list form for further evaluation.
        final, raw = predict(prior, post, cleaned_data_test)

    else:   
        # same steps as above but for 'holdout' partitioning.
        dic,n_instances,cleaned_data,n_attrs = preprocess(file, split=holdout_split)
        prior,post,_ = train(cleaned_data, n_instances, dic, n_attrs)
        final, raw = predict(prior, post, cleaned_data)
    # get a baseline metric based on the majority class for the dataset (zero rule method)
    baseline = max(dic.values())/n_instances
    hits=0
    accuracy=0
 
    recall = defaultdict(int)
    prec = defaultdict(int)
    # if we have a hit then this is a true positive.
    for cls, instance in zip(final.values(), raw):
        if cls[0]==instance[-1]:
            hits+=1
            recall[cls[0]]+=1
        else:
        #elif (instance[-1]==key):
        #    other[key]+=1
        # otherwise, we have a FN or FP.
            prec[cls[0]]+=1
    # determines overall accuracy of the classifier
    accuracy = hits/n_instances
    final_metrics = defaultdict(int)
    # utility function to sort the values in lexicographical order.
    sorter_func = lambda x: x[0]
    # iterate through our stored classes to obtain metrics for each class in the dataset.
    for (a,b), (_, d), (_,f) in zip(sorted(recall.items(), key = sorter_func), sorted(prec.items(), key = sorter_func), 
                                sorted(dic.items(), key = sorter_func)):
        # 'p' holds the precision metric (TP/TP+FP), 'r' holds the recall metric (TP/TP+FN)
        p,r = b/(b+d),  b/(b+f)
        # 'fs' is the f1-score, the harmonic mean of 'p' and 'r'.
        fs = 2*((p*r)/(p+r))
        # store the values in order in a dictionary of tuples.
        final_metrics[a] = (p, r, fs)
    
    return accuracy, baseline, final_metrics

In [583]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain(file):
    """Determines the information gain attribute for each attribute found in the csv file argument and returns
    the attribute found with the highest information gain.
    
    Args:
        file (csv file): dataset
    
    Returns:
        ig (dic): a dictionary containing each attribute in the dataset and their respective information gain, 
        sorted in descending order.
        best_attr(key: value): the attribute that had the maximum information gain the dataset.
    """
    # preprocess the file to get the number of attributes and instances found in the dataset.
    dic,n_instances,cleaned_data,n_attrs = preprocess(file)
    # obtain a dictionary (ig_dic) of conditional counts of each attribute found in the dataset i.e. given each
    # class, how many instances of each attribute's values are found for a given class.
    prior,ig_dic,_= train(cleaned_data, n_instances, dic, n_attrs, ig_dic=True)
    # determine the class entropy (H(R)) using the raw counts from the prior probs.
    class_entropy = -(np.sum([g*np.log2(g) for g in list(prior.values())]))
    # Determine the raw counts of each attribute, regardless of their class.
    attr_dic= defaultdict(int)
    for i in range(n_attrs):
        attr_dic[i] = defaultdict(int)
        for line in cleaned_data:
            attr_dic[i][line[i]]+=1
    ar_orig = copy.deepcopy(attr_dic)  
    # determine the probability of each attribute's value occurring in the dataset.
    probs_mean_info = attr_dic.copy()
    for key, ar in attr_dic.items():
        probs_mean_info[key] = ar
        for key2, val in ar.items():
            probs_mean_info[key][key2] = val/n_instances
    
    
    def log_probs(attr_dic, ig_dic, probs_mean_info, attr):
        """Determines the information gain for each attribute found in the dataset.
    
        Args:
            attr_dic (dic): dictionary containing the raw counts for each attribute found in the dataset.
            ig_dic (dic): dictionarying containing the conditional counts for each attribute's values found in the dataset.
            probs_mean_info (dic): dictionary containing the (prior) probabilities of finding a given attribute in
            the dataset.
            attr (number): the attribute which will be evaluated to determine the mean information.
            This is an integer corresponding to a key found in the attribute dictionary.
    
        Returns:
            mean_info (numeric value): the 'mean information' for the attribute.
        """
        # sanity check to ensure the user has inputted a valid key.
        assert attr in attr_dic.keys()
        # dictionary to contain the entropies for the attribute's values. 
        logs_dic = defaultdict(int)
        mean_info = 0
        # iterate through the attribute dictionary, only looking at matching keys to our input attribute.
        for a, b in attr_dic[attr].items():
            for d in ig_dic[attr].values():
                for g in d.items():
                    # if the keys equal each other, then we multiply the prior prob of the attribute's value
                    # by the log of the prior prob of the attribute's value: sum from 1 to the number of types of attr vals
                    #where (attr='value') = -P(attr=val|class)*log(P(attr=val|class))
                    if a == g[0]:
                        logs_dic[a] += -(g[1]/b)*np.log2(g[1]/b)
        # the mean information is obtained by multiplying the entropies of the attribute's values by their respective
        # prior probabilties of being found in the dataset.
        for (a,b),(c,d) in zip(sorted(logs_dic.items()), sorted(probs_mean_info[attr].items())):
            mean_info+=b*d
        return mean_info
    
    # iterate through each attribute to find the 'information gain' for each attribute. We subtract the 'mean information'
    # for each attribute from H(R).
    ig = defaultdict(int)
    for i in range(n_attrs):
        ig_attr = class_entropy - log_probs(ar_orig, ig_dic, probs_mean_info, i)
        ig['attribute: ' + str(i)] = ig_attr
    # obtain the attribute which has the greatest information gain.
    best_attr = max(ig.items(), key=operator.itemgetter(1))
    
    # sort the attributes according to the information gain in descending order and return the attribute with the greatest
    # information gain.
    return sorted(ig.items(), reverse=True, key = lambda x: x[1]), best_attr
        

In [561]:
a= info_gain('hypothyroid.csv')
a

([('attribute: 12', 0.0093537102155803464),
  ('attribute: 14', 0.005792553705846859),
  ('attribute: 15', 0.0057682882016146242),
  ('attribute: 16', 0.0057440312456027987),
  ('attribute: 13', 0.0040754934196237658),
  ('attribute: 17', 0.0025804275555744161),
  ('attribute: 5', 0.0013683791752742147),
  ('attribute: 2', 0.0012382074503017315),
  ('attribute: 4', 0.00099852939063360679),
  ('attribute: 1', 0.00091393511608500733),
  ('attribute: 9', 0.00089830040440280756),
  ('attribute: 6', 0.00054230064444243942),
  ('attribute: 8', 0.00048887576912848285),
  ('attribute: 0', 0.00044766932839335194),
  ('attribute: 7', 0.00043509384646389648),
  ('attribute: 3', 0.00014844815831743796),
  ('attribute: 11', 7.8684698479491999e-05),
  ('attribute: 10', 4.4637788243040433e-05)],
 ('attribute: 12', 0.0093537102155803464))

In [584]:
def main(DIR):
# 'dir' argument should specify directory containg csv datasets. This function runs all the functions found in the pipeline
# to give evaluation metrics.
    
    count=0
    total=0.0
    avg = 0.0
    total_h=0.0
    avg_h=0.0
    for file in os.listdir(DIR):
        if str(file).endswith('.csv'):
            count+=1
            acc, base, c = evaluate(file)
            acc_h,_, cd = evaluate(file, holdout_split=0.8)
            _, best_attr = info_gain(file)
            total+=acc
            total_h+=acc_h
            print("TEST ACCURACY for", str(file).upper(), "is", acc, " \n and for BASELINE is : ", base, " \n and HOLDOUT is: ", acc_h)
            print("ATTRIBUTE with GREATEST INFORMATION GAIN is: \n", best_attr)
            print("RELATIVE CLASSIFIER performance compared to BASELINE is:", 100*(acc - base), "% DIFFERENCE")
            print("HOLDOUT performance compared to BASELINE is:", 100*(acc_h - base), "% DIFFERENCE")
            print("PRECISION, RECALL and F1-score for each class are: \n ", c.items())
            print("PRECISION, RECALL and F1-score for HOLDOUT for each class are: \n", cd.items())
            print()
            avg=total/count
            avg_h=total_h/count
    print("MEAN ACCURACY is", avg, "and for HOLDOUT it is: ", avg_h)

In [585]:
main('.')

TEST ACCURACY for ANNEAL.CSV is 0.9910913140311804  
 and for BASELINE is :  0.7616926503340757  
 and HOLDOUT is:  0.9888888888888889
ATTRIBUTE with GREATEST INFORMATION GAIN is: 
 ('attribute: 11', 0.43517783626288553)
RELATIVE CLASSIFIER performance compared to BASELINE is: 22.939866369710472 % DIFFERENCE
HOLDOUT performance compared to BASELINE is: 22.71962385548132 % DIFFERENCE
PRECISION, RECALL and F1-score for each class are: 
  dict_items([('1', (0.6153846153846154, 0.5, 0.5517241379310345)), ('2', (0.98989898989899, 0.49746192893401014, 0.6621621621621622)), ('3', (0.9970544918998527, 0.49742836149889785, 0.6637254901960784))])
PRECISION, RECALL and F1-score for HOLDOUT for each class are: 
 dict_items([('1', (0.3333333333333333, 0.125, 0.18181818181818182))])

TEST ACCURACY for BREAST-CANCER.CSV is 0.7552447552447552  
 and for BASELINE is :  0.7027972027972028  
 and HOLDOUT is:  0.7931034482758621
ATTRIBUTE with GREATEST INFORMATION GAIN is: 
 ('attribute: 5', 0.07700985251

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.


An interesting relationship can be seen between the attributes with the maximal information gain, and the overall accuracy metric, when there was a disparity between the accuracy and the ZeroR benchmark. For example, in 'hypothroid.csv' 95% of its instances were of class 'negative'. The classifier performed the same as the ZeroR benchmark in this instance and was the only example of where the classifier perfomed the same as the benchmark.
The attribute with the largest information gain for that dataset had an information gain of 0.009, indicating that there was little information to be gained from individual attributes that would help discern the classifier between the two classes in this dataset. This may also indicate that their wasn't much correlation between the various attributes and the two classes and shows how misleading the accuracy benchmark can be, as the classifier obtained a 95% class accuracy. This could be due to a large class imbalance between the large number of negative instances and small number of 'hypothroid' instances, and if the attributes don't correlate much to the predictions, then the classifier is probably using the large prior probability of the 'negative' instances to make predictions, and using little of the posterior probabilities extracted from Bayes' formula in making predictions.
Likewise, for the dataset 'breast_cancer.csv', there was little difference
between the benchmark and the classifier's performance, and the information gain for each attribute was extremely low.
Conversely, when the classifier performed much better than the benchmark, such as in 'nursery.csv', the attribute with the largest information gain (attribute 7, corresponding to 'health) had quite a large value (0.96 information gain).
This is true of other datasets which performed much better than the benchmark such as the 'mushroom.csv' dataset. 
This shows a strong positive correlation between a well-performing classifier and an attribute/s that have a large information gain.
It seems that the classifier was performing much better than the baseline benchmark when there were attributes that were strongly correlated with particular classes, which is indicated by the relatively linear relationship between large information gain attributes and effective classifiers. This suggests that for those kinds of datasets, some of the attributes were effective
in contributing to a discriminative effect between the classes.

4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

Using a holdout strategy of an 80% training, 20% test split, the overall accuracy was surprisingly similar to results when training on the full model (c.f. above cells where MEAN ACCURACY is 0.8260886310750691 and for HOLDOUT it is:  0.8195149616014783).
Even with varying the holdout partitions to smaller training partitions, there wasn't a sharp difference in the accuracy.
For example, with using a training partition of only 30% of the data, the overall accuracy was only approximately 4% lower than training on the full dataset.
Addditionally, there were individual datasets where the holdout classifier outperformed the classifier trained on the full dataset (hepatitis.csv, breast-cancer.csv).
These results are quite surprising and counter-intuitive, owing to the inherent bias with training on a full dataset. One would also assume that training on a partition that consisted of a small fraction of the data could mean the classifier wouldn't have enough data to infer solid enough relationships between attributes and different classes, or that there may be class imbalances present that would mean the classifier wouldn't be able to infer strong correlations between individual attributes and the classes. 
The fact that the holdout strategy, even when used on small training partitions, performed quite similarly to the full classifier may suggest that there were fairly uniform "attribute=val|class" distributions in *most* of the datasets, such that the posterior probabilities were consistent amongst random partitions. For example, P(attr(x)=1|class(A))=0.95, P(attr(x)=2|class(B))=0.9, so that even with a relatively smaller training partition, testing accuracy is similar to training on the full dataset/s as there were attributes in those datasets that were effective at discriminating between classes (evidenced by large information gain).
Even with datasets where there was evidently nittle discriminatory information found in all the attributes (such as hypothroid.csv), the holdout strategy performed similarly to the full classifier as well as the ZeroR benchmark, probably because heavy prediction weights were attributed to the prior probabilities of the classes and there were huge class imbalances in those datasets


In [570]:
a,b = info_gain('primary-tumor.csv')

In [571]:
a

[('attribute: 2', 0.55235262338090285),
 ('attribute: 3', 0.37910426027718858),
 ('attribute: 1', 0.3353600515055537),
 ('attribute: 12', 0.29153013602249223),
 ('attribute: 14', 0.24588868143377063),
 ('attribute: 8', 0.22052193470670511),
 ('attribute: 4', 0.21246189904816459),
 ('attribute: 9', 0.19976143639025112),
 ('attribute: 15', 0.18425767171538388),
 ('attribute: 16', 0.1701481108388716),
 ('attribute: 0', 0.15474214188705782),
 ('attribute: 13', 0.12715354518198163),
 ('attribute: 6', 0.10088123982398978),
 ('attribute: 7', 0.067872775704422406),
 ('attribute: 10', 0.067144602410105225),
 ('attribute: 11', 0.06025390884525228),
 ('attribute: 5', 0.020366938848049188)]